<a href="https://colab.research.google.com/github/scaairesearch/da_cv_fer/blob/main/Testing_Synthetic_Data_2503.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/scaairesearch/da_cv_fer.git
import os
os.getcwd()
os.chdir('da_cv_fer')

Cloning into 'da_cv_fer'...
remote: Enumerating objects: 370, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 370 (delta 79), reused 67 (delta 32), pack-reused 209
Receiving objects: 100% (370/370), 26.43 MiB | 16.32 MiB/s, done.
Resolving deltas: 100% (193/193), done.


In [ ]:
!pip install -q -r requirements.txt # -q is for quiet install
# for some reason the below does not get installed with requirements
!pip install -q opendatasets
!pip install -q facenet_pytorch

In [3]:
#@title loading libraries
from da_cv_fer.data_config import DataConfig
from da_cv_fer.ds_sfew import DatasetSFEW
from da_cv_fer.ds_sfew_crop import DatasetSFEWCROP
from da_cv_fer.ds_expw_crop import EXPWCROP
from da_cv_fer.ds_expw import EXPW
from da_cv_fer.utils import *
from da_cv_fer.model_DANN import DANN
from da_cv_fer.model_DANN_Bespoke import DANNBespoke
from da_cv_fer.run_config import RunConfig
from da_cv_fer.train import *
import os
import zipfile

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from tqdm import tqdm # for beautiful model training updates

import matplotlib.pyplot as plt # for plots

import time
from pathlib import Path

from copy import deepcopy
import numpy as np


# importing libraries
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import collections
import cv2
import os

In [22]:
import warnings

# Ignore all warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cpu


In [5]:
#@title Loading models to evaluate
# non_dann_model_sfew_name =  'non_dann_sfew_expw_14_03.pt'
dataconfig_inference = DataConfig()

In [24]:
# non_dann_model_inference = DANNBespoke(num_classes=7, backbone='mobilenet', attention=False, dropout=0)
# model_dir = Path('/content/drive/MyDrive/SCAAI_FER/Models/non_dann_sfew_expw_14_03.pt')
model_dates = ['03_03','14_03']
inference_models ={}
for model_date in model_dates:
  model_name = f'model_{model_date}'
  model_path = f'/content/drive/MyDrive/CV_FER/models/non_dann_sfew_expw/non_dann_sfew_expw_{model_date}.pt'
  try:
    inference_models[model_name] = DANNBespoke(num_classes=7, backbone='mobilenet', attention=False, dropout=0)
    inference_models[model_name].load_state_dict(torch.load(model_path, map_location=torch.device(device)))
    print(f'{model_name} loaded successfully')
  except Exception as e:
    print(f'Problem in loading {model_name}: {e}')


# model_dir = '/content/drive/MyDrive/CV_FER/models/non_dann_sfew_expw/non_dann_sfew_expw_14_03.pt'



# try:
#     non_dann_model_inference.load_state_dict(torch.load(model_dir, map_location=torch.device(device)))
#     print(f'{model_dir} loaded successfully')
# except Exception as e:
#     print(f'Problem in loading {model_dir}: {e}')


feature extractor backbone created using mobilenet model
model_03_03 loaded successfully
feature extractor backbone created using mobilenet model
model_14_03 loaded successfully


In [27]:
inference_models.keys()

dict_keys(['model_03_03', 'model_14_03'])

In [10]:
# Define a mapping from numerical index to emotion label
labels_map = {
    "0": "Angry",
    "1": "Disgust",
    "2": "Fear",
    "3": "Happy",
    "4": "Sad",
    "5": "Surprise",
    "6": "Neutral"
}

In [13]:
import os

def get_files_in_folder(folder_path):
    files = []
    for file in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file)):
            files.append(file)
    return files


In [ ]:
#@title Cropped Images
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
import IPython.display as display
from facenet_pytorch import MTCNN


synthetic_image_base_path = '/content/drive/MyDrive/CV_FER/dataset'
dict_synthetic_image_folder = {'1':("synthetic_test_data_tranche_1","synthetic_test_data_tranche_1_cropped"),
                               "2":("synthetic_test_data_tranche_2","synthetic_test_data_tranche_2_cropped")}

# initiatate MTCNN
mtcnn = MTCNN(image_size=224)

df_columns = ['image_name','cropped_image','tranche','emotion_gt','emotion_03_03','emotion_14_03']
image_df = pd.DataFrame(columns=df_columns)

for tranche, (orig_folder,cropped_folder) in dict_synthetic_image_folder.items():
  orig_path = os.path.join(synthetic_image_base_path, orig_folder)
  files_list = get_files_in_folder(orig_path)
  # print(files_list)
  for filename in files_list:
    image_path = os.path.join(orig_path, filename)
    img = Image.open(image_path).convert('RGB')

    #cropped
    img_cropped = mtcnn(img,
                        save_path=os.path.join(synthetic_image_base_path, cropped_folder, filename))
    # Scale the values from [-1, 1] to [0, 1]
    normalized_image_cropped = (img_cropped + 1) / 2
    normalized_image_cropped = torch.clamp(normalized_image_cropped, 0, 1)

    emotion_gt_label = filename.split('_')[0]

    column_values ={
        'image_name':filename,
        'cropped_image':normalized_image_cropped,
        'tranche':tranche,
        'emotion_gt':emotion_gt_label,
        'emotion_03_03':9,
        'emotion_14_03':9
    }
    image_df = image_df.append(column_values, ignore_index=True)

image_df.head()

In [ ]:
image_df

In [38]:
## inferencing


for inference_model_name, inference_model in inference_models.items():
    print(f"Inference Model: {inference_model_name}")
    model_date = ('_').join(inference_model_name.split("_")[-2:])
    print(f"Model Date: {model_date}")
    column_name = f'emotion_{model_date}'
    print(f"Column Name: {column_name}")

    inference_model.eval()  # Set the model to evaluation mode

    for index, row in image_df.iterrows():
        image = row['cropped_image']
        image = image.to(device)
        with torch.no_grad():
          image = image.unsqueeze(0) # batch  = 1
          output,_,_ = inference_model(image)
          _, predicted_index = torch.max(output, 1)
          predicted_label = labels_map[str(predicted_index.item())]  # Convert numerical index to emotion label
          print(f"Predicted Emotion: {predicted_label}")
          print("Ground Truth:",row['emotion_gt'])
          df_row = image_df.loc[index]
          df_row[column_name] = predicted_label
          image_df.loc[index] = df_row

print("Inferencing complete")

Inference Model: model_03_03
Model Date: 03_03
Column Name: emotion_03_03
Predicted Emotion: Fear
Ground Truth: Surprised
Predicted Emotion: Fear
Ground Truth: Surprised
Predicted Emotion: Fear
Ground Truth: Sad
Predicted Emotion: Fear
Ground Truth: Sad
Predicted Emotion: Fear
Ground Truth: Neutral
Predicted Emotion: Happy
Ground Truth: Happy
Predicted Emotion: Sad
Ground Truth: Neutral
Predicted Emotion: Happy
Ground Truth: Happy
Predicted Emotion: Fear
Ground Truth: Disgust
Predicted Emotion: Disgust
Ground Truth: Angry
Predicted Emotion: Fear
Ground Truth: Fear
Predicted Emotion: Fear
Ground Truth: Fear
Predicted Emotion: Fear
Ground Truth: Disgust
Predicted Emotion: Disgust
Ground Truth: Angry
Predicted Emotion: Fear
Ground Truth: Angry
Predicted Emotion: Happy
Ground Truth: Fear
Predicted Emotion: Surprise
Ground Truth: Fear
Predicted Emotion: Sad
Ground Truth: Fear
Predicted Emotion: Fear
Ground Truth: Fear
Predicted Emotion: Sad
Ground Truth: Fear
Predicted Emotion: Fear
Ground 

In [39]:
image_df

,image_name,cropped_image,tranche,emotion_gt,emotion_03_03,emotion_14_03
0,Surprised_Child_African.png,"[[[tensor(0.0059), tensor(0.0059), tensor(0.00...",1,Surprised,Fear,Angry
1,Surprised_Male_Asian.png,"[[[tensor(0.6543), tensor(0.6543), tensor(0.65...",1,Surprised,Fear,Neutral
2,Sad_Male_Indian.png,"[[[tensor(0.1738), tensor(0.1738), tensor(0.18...",1,Sad,Fear,Fear
3,Sad_Female_Tilt.png,"[[[tensor(0.3848), tensor(0.3770), tensor(0.40...",1,Sad,Fear,Surprise
4,Neutral_Woman_Jewels.png,"[[[tensor(0.6230), tensor(0.5449), tensor(0.49...",1,Neutral,Fear,Surprise
...,...,...,...,...,...,...
96,Surprise_Female_Hijab_India.png,"[[[tensor(0.6426), tensor(0.6426), tensor(0.63...",2,Surprise,Angry,Disgust
97,Surprise_Male_Beard_European.png,"[[[tensor(0.1699), tensor(0.1426), tensor(0.16...",2,Surprise,Angry,Neutral
98,Surprise_Male_Makeup_African.png,"[[[tensor(0.2520), tensor(0.2520), tensor(0.24...",2,Surprise,Sad,Surprise
99,Surprise_Male_Hat_White.png,"[[[tensor(0.1543), tensor(0.1699), tensor(0.14...",2,Surprise,Sad,Surprise


from matplotlib import pyplot as plt
import seaborn as sns
image_df.groupby('tranche').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
image_df.groupby('emotion_gt').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
image_df.groupby('emotion_03_03').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
image_df.groupby('emotion_14_03').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['emotion_gt'].value_counts()
    for x_label, grp in image_df.groupby('tranche')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('tranche')
_ = plt.ylabel('emotion_gt')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['emotion_03_03'].value_counts()
    for x_label, grp in image_df.groupby('emotion_gt')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('emotion_gt')
_ = plt.ylabel('emotion_03_03')

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['emotion_14_03'].value_counts()
    for x_label, grp in image_df.groupby('emotion_03_03')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('emotion_03_03')
_ = plt.ylabel('emotion_14_03')

In [40]:
results_df = image_df.drop(columns=['cropped_image'])
results_df.head()

,image_name,tranche,emotion_gt,emotion_03_03,emotion_14_03
0,Surprised_Child_African.png,1,Surprised,Fear,Angry
1,Surprised_Male_Asian.png,1,Surprised,Fear,Neutral
2,Sad_Male_Indian.png,1,Sad,Fear,Fear
3,Sad_Female_Tilt.png,1,Sad,Fear,Surprise
4,Neutral_Woman_Jewels.png,1,Neutral,Fear,Surprise


In [58]:
results_df.to_excel('results.xlsx', index=False)

In [45]:
# Creating the Metrics
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, classification_report


def print_metrics(true_labels, predicted_labels):
  # Compute accuracy
  accuracy = accuracy_score(true_labels, predicted_labels)
  print("Accuracy:", accuracy)

  # Compute precision
  precision = precision_score(true_labels, predicted_labels, average='weighted')
  print("Precision:", precision)

  # Generate confusion matrix
  conf_matrix = confusion_matrix(true_labels, predicted_labels)
  print("Confusion Matrix:")
  print(conf_matrix)

  # Generate classification report
  class_report = classification_report(true_labels, predicted_labels)
  print("Classification Report:")
  print(class_report)


In [50]:
predicted_labels_03_03_1 = results_df['emotion_03_03'][image_df['tranche'] == '1'].tolist()
predicted_labels_14_03_1 =  results_df['emotion_14_03'][image_df['tranche'] == '1'].tolist()

predicted_labels_03_03_2  = results_df['emotion_03_03'][image_df['tranche'] == '2'].tolist()
predicted_labels_14_03_2  =  results_df['emotion_14_03'][image_df['tranche'] == '2'].tolist()

true_labels_1 = results_df['emotion_gt'][image_df['tranche'] == '1'].tolist()
true_labels_2 = results_df['emotion_gt'][image_df['tranche'] == '2'].tolist()

predicted_labels_03_03 = results_df['emotion_03_03'].tolist()
predicted_labels_14_03 = results_df['emotion_14_03'].tolist()

true_labels = results_df['emotion_gt'].tolist()

In [51]:
# Tranche 1 : 03_03 v GT
print_metrics(true_labels_1, predicted_labels_03_03_1)

Accuracy: 0.2857142857142857
Precision: 0.17460317460317462
Confusion Matrix:
[[0 2 0 0 0 0 0]
 [0 0 2 0 0 0 0]
 [0 0 2 0 0 0 0]
 [0 0 0 2 0 0 0]
 [0 0 1 0 0 1 0]
 [0 0 2 0 0 0 0]
 [0 0 2 0 0 0 0]]
Classification Report:
              precision    recall  f1-score   support

       Angry       0.00      0.00      0.00         2
     Disgust       0.00      0.00      0.00         2
        Fear       0.22      1.00      0.36         2
       Happy       1.00      1.00      1.00         2
     Neutral       0.00      0.00      0.00         2
         Sad       0.00      0.00      0.00         2
   Surprised       0.00      0.00      0.00         2

    accuracy                           0.29        14
   macro avg       0.17      0.29      0.19        14
weighted avg       0.17      0.29      0.19        14



In [52]:
# Tranche 1 : 14_03 v GT
print_metrics(true_labels_1, predicted_labels_14_03_1)

Accuracy: 0.21428571428571427
Precision: 0.21428571428571427
Confusion Matrix:
[[1 1 0 0 0 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 0 2 0 0 0]
 [0 0 0 2 0 0 0 0]
 [0 0 0 0 0 1 1 0]
 [0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]
 [1 0 0 0 1 0 0 0]]
Classification Report:
              precision    recall  f1-score   support

       Angry       0.50      0.50      0.50         2
     Disgust       0.00      0.00      0.00         2
        Fear       0.00      0.00      0.00         2
       Happy       1.00      1.00      1.00         2
     Neutral       0.00      0.00      0.00         2
         Sad       0.00      0.00      0.00         2
    Surprise       0.00      0.00      0.00         0
   Surprised       0.00      0.00      0.00         2

    accuracy                           0.21        14
   macro avg       0.19      0.19      0.19        14
weighted avg       0.21      0.21      0.21        14



In [53]:
# Tranche 2 : 03_03 v GT
print_metrics(true_labels_2, predicted_labels_03_03_2)

Accuracy: 0.3103448275862069
Precision: 0.2557535426863921
Confusion Matrix:
[[ 4  3  4  0  1  1  1]
 [ 3  1  3  3  0  2  1]
 [ 0  0  5  1  0  3  3]
 [ 0  0  0 11  0  1  1]
 [ 0  2  1  4  0  2  1]
 [ 2  4  3  0  0  1  3]
 [ 2  0  2  0  0  3  5]]
Classification Report:
              precision    recall  f1-score   support

       Angry       0.36      0.29      0.32        14
     Disgust       0.10      0.08      0.09        13
        Fear       0.28      0.42      0.33        12
       Happy       0.58      0.85      0.69        13
     Neutral       0.00      0.00      0.00        10
         Sad       0.08      0.08      0.08        13
    Surprise       0.33      0.42      0.37        12

    accuracy                           0.31        87
   macro avg       0.25      0.30      0.27        87
weighted avg       0.26      0.31      0.28        87



In [54]:
# Tranche 2 : 14_03 v GT
print_metrics(true_labels_2, predicted_labels_14_03_2)

Accuracy: 0.3563218390804598
Precision: 0.3447229082187465
Confusion Matrix:
[[ 3  8  1  0  2  0  0]
 [ 1  9  0  0  2  0  1]
 [ 0  3  1  0  3  3  2]
 [ 1  1  0 10  0  1  0]
 [ 2  1  0  1  3  2  1]
 [ 3  5  1  0  0  0  4]
 [ 0  2  0  1  3  1  5]]
Classification Report:
              precision    recall  f1-score   support

       Angry       0.30      0.21      0.25        14
     Disgust       0.31      0.69      0.43        13
        Fear       0.33      0.08      0.13        12
       Happy       0.83      0.77      0.80        13
     Neutral       0.23      0.30      0.26        10
         Sad       0.00      0.00      0.00        13
    Surprise       0.38      0.42      0.40        12

    accuracy                           0.36        87
   macro avg       0.34      0.35      0.32        87
weighted avg       0.34      0.36      0.33        87



In [55]:
# All : 03_03 v GT
print_metrics(true_labels, predicted_labels_03_03)

Accuracy: 0.3069306930693069
Precision: 0.2480688545044981
Confusion Matrix:
[[ 4  5  4  0  1  1  1  0]
 [ 3  1  5  3  0  2  1  0]
 [ 0  0  7  1  0  3  3  0]
 [ 0  0  0 13  0  1  1  0]
 [ 0  2  2  4  0  3  1  0]
 [ 2  4  5  0  0  1  3  0]
 [ 2  0  2  0  0  3  5  0]
 [ 0  0  2  0  0  0  0  0]]
Classification Report:
              precision    recall  f1-score   support

       Angry       0.36      0.25      0.30        16
     Disgust       0.08      0.07      0.07        15
        Fear       0.26      0.50      0.34        14
       Happy       0.62      0.87      0.72        15
     Neutral       0.00      0.00      0.00        12
         Sad       0.07      0.07      0.07        15
    Surprise       0.33      0.42      0.37        12
   Surprised       0.00      0.00      0.00         2

    accuracy                           0.31       101
   macro avg       0.22      0.27      0.23       101
weighted avg       0.25      0.31      0.27       101



In [56]:
# All : 14_03 v GT
print_metrics(true_labels, predicted_labels_14_03)

Accuracy: 0.33663366336633666
Precision: 0.3187175860443187
Confusion Matrix:
[[ 4  9  1  0  2  0  0  0]
 [ 1  9  0  0  4  0  1  0]
 [ 0  3  1  0  5  3  2  0]
 [ 1  1  0 12  0  1  0  0]
 [ 2  1  0  1  3  3  2  0]
 [ 3  5  2  0  0  0  5  0]
 [ 0  2  0  1  3  1  5  0]
 [ 1  0  0  0  1  0  0  0]]
Classification Report:
              precision    recall  f1-score   support

       Angry       0.33      0.25      0.29        16
     Disgust       0.30      0.60      0.40        15
        Fear       0.25      0.07      0.11        14
       Happy       0.86      0.80      0.83        15
     Neutral       0.17      0.25      0.20        12
         Sad       0.00      0.00      0.00        15
    Surprise       0.33      0.42      0.37        12
   Surprised       0.00      0.00      0.00         2

    accuracy                           0.34       101
   macro avg       0.28      0.30      0.27       101
weighted avg       0.32      0.34      0.31       101



Later part of the notebook not used